# 라이브러리 불러오기 및 폴리움 설치

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import folium
import requests
import json

# 서울시 상권 시각화

In [4]:
df1 = pd.read_csv('/content/drive/MyDrive/DSL/EDA프로젝트/상권변화2022-03.csv')
df1.drop(0, inplace = True)
df_all = df1.iloc[:,[2,6]]

In [5]:
# 상권변화 지표 encoding

encoding_list = []

for change in list(df_all['상권변화지표']):
  if change == '정체':
    encoding_list.append(1)
  elif change == '상권축소':
    encoding_list.append(0)
  elif change == '다이나믹':
    encoding_list.append(2)
  else:
    encoding_list.append(3)

df_all['encoding'] = encoding_list
encoding_list

<ipython-input-5-7ba1c2e7ac32>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['encoding'] = encoding_list


[1, 1, 1, 3, 2, 1, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1, 2, 2, 2]

In [6]:
# 서울 행정구역 json raw파일(githubcontent)

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [7]:
# 서울시 상권 시각화

m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

folium.Choropleth(
    geo_data=seoul_geo,
    data=df_all, 
    columns=('행정구역', 'encoding'), 
    key_on='properties.name',
    fill_color='YlGn', 
    legend_name='상권변화',
).add_to(m)
m

# 성동구, 성북구 상권 시각화

In [8]:
sb_df = pd.read_csv('/content/drive/MyDrive/DSL/EDA프로젝트/상권변화_성북구.csv')
sd_df = pd.read_csv('/content/drive/MyDrive/DSL/EDA프로젝트/상권변화_성동구.csv')

In [9]:
sb_df.drop(0,inplace = True)

In [10]:
sb_change_df = sb_df[(sb_df['년도'] == 2022) & (sb_df['분기'] == 3)]
sd_change_df = sd_df[(sd_df['년도'] == 2022) & (sd_df['분기'] == 3)]

In [11]:
# 상권변화 지표 encoding

encoding_list = []

for change in list(sd_change_df['상권변화지표']):
  if change == '정체':
    encoding_list.append(2)
  elif change == '상권축소':
    encoding_list.append(1)
  elif change == '다이나믹':
    encoding_list.append(2)
  else:
    encoding_list.append(3)

sd_change_df['encoding'] = encoding_list

encoding_list = []

for change in list(sb_change_df['상권변화지표']):
  if change == '정체':
    encoding_list.append(2)
  elif change == '상권축소':
    encoding_list.append(1)
  elif change == '다이나믹':
    encoding_list.append(2)
  else:
    encoding_list.append(3)

sb_change_df['encoding'] = encoding_list


<ipython-input-11-558885334ee2>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sd_change_df['encoding'] = encoding_list
<ipython-input-11-558885334ee2>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb_change_df['encoding'] = encoding_list


In [12]:
# 생존률 파일 불러온 후 전처리 및 merge

sb_life_df = pd.read_csv('/content/drive/MyDrive/DSL/EDA프로젝트/성북-신생기업생존율.csv')
sd_life_df = pd.read_csv('/content/drive/MyDrive/DSL/EDA프로젝트/동별_신생_생존율.csv')

sb_now_life_df = sb_life_df.iloc[:,0:10:3]
sd_now_life_df = sd_life_df.iloc[:,0:10:3]
sb_now_life_df.drop(0, inplace = True)

rename_list = []

for place in list(sb_now_life_df['행정동']):
  rename_list.append('성북구 ' + place)
sb_now_life_df['행정동'] = rename_list

sb_life_df = sb_now_life_df.iloc[:, [0,3]]
sd_life_df = sd_now_life_df.iloc[:,[0,3]]
sd_life_df.rename(columns = {'행정구역' : '행정동'}, inplace = True)
sd_life_df.drop(17, inplace = True)

sd_change_df.rename(columns = {'행정구역':'행정동'}, inplace = True)
sb_change_df.rename(columns = {'행정구역':'행정동'}, inplace = True)

sd_data = pd.merge(sd_life_df, sd_change_df, how = 'left', on = '행정동')
sb_data = pd.merge(sb_life_df, sb_change_df, how = 'left', on = '행정동')
sd_data['final_encoding'] = sd_data['2022_3분기_3년생존율']*sd_data['encoding']
sb_data['final_encoding'] = sb_data['22_3_3']*sb_data['encoding']

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
# 전국 행정동 단위 json 파일

r = requests.get('https://raw.githubusercontent.com/vuski/admdongkor/master/ver20230101/HangJeongDong_ver20230101.geojson')
c = r.content
geo_data = json.loads(c)

In [14]:
# 성북구, 성동구 json 파일 추출

sb_js_list = []
sd_js_list = []

for i in range(0, 450):
  if geo_data['features'][i]['properties']['sggnm'] == '성북구':
    sb_js_list.append(geo_data['features'][i])
  if geo_data['features'][i]['properties']['sggnm'] == '성동구':
    sd_js_list.append(geo_data['features'][i])

sb_json = {
"type": "FeatureCollection",
"name": "HangJeongDong_ver20230101",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": sb_js_list}

sd_json = {
"type": "FeatureCollection",
"name": "HangJeongDong_ver20230101",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": sd_js_list}

In [15]:
sd_json['features'][15]['properties']['temp'] = '성동구 금호2.3가동' # json 파일 수정

In [21]:
# 성동구 상권확장 지역 시각화

m = folium.Map(
    location=[37.5536588726331, 127.042026826884],
    zoom_start=14, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    sd_json,
    name='temp'
).add_to(m)

folium.Choropleth(
    geo_data=sd_json,
    data=sd_data, 
    columns=('행정동', 'encoding'), 
    key_on='properties.temp',
    fill_color='YlGn', 
    legend_name='상권변화',
).add_to(m)
m

In [20]:
# 성동구 상권확장 & 생존률 시각화

m = folium.Map(
    location=[37.5536588726331, 127.042026826884],
    zoom_start=14, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    sd_json,
    name='temp'
).add_to(m)

folium.Choropleth(
    geo_data=sd_json,
    data=sd_data, 
    columns=('행정동', 'final_encoding'), 
    key_on='properties.temp',
    fill_color='YlGn', 
    legend_name='상권변화',
).add_to(m)
m

In [18]:
# 성북구 상권확장 지역 시각화

m = folium.Map(
    location=[37.601394911855, 127.02168184887],
    zoom_start=13, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    sb_json,
    name='temp'
).add_to(m)

folium.Choropleth(
    geo_data=sb_json,
    data=sb_data, 
    columns=('행정동', 'encoding'), 
    key_on='properties.temp',
    fill_color='YlGn', 
    legend_name='상권변화',
).add_to(m)
m

In [19]:
# 성북구 상권확장 & 생존률 시각화

m = folium.Map(
    location=[37.601394911855, 127.02168184887],
    zoom_start=13, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    sb_json,
    name='temp'
).add_to(m)

folium.Choropleth(
    geo_data=sb_json,
    data=sb_data, 
    columns=('행정동', 'final_encoding'), 
    key_on='properties.temp',
    fill_color='YlGn', 
    legend_name='상권변화',
).add_to(m)
m